In [26]:
import numpy as np
import matplotlib.pyplot as plt
from Regler_class_file import PI_controller_class

#importing Druckrohrleitung
import sys
import os
current = os.path.dirname(os.path.realpath('Main_Programm.ipynb'))
parent = os.path.dirname(current)
sys.path.append(parent)
from functions.pressure_conversion import pressure_conversion
from Ausgleichsbecken.Ausgleichsbecken_class_file import Ausgleichsbecken_class
from Turbinen.Turbinen_class_file import Francis_Turbine

In [27]:
#define constants

#Turbine
Q_nenn = 0.85
p_nenn,_ = pressure_conversion(10.6,'bar','Pa')

# physics
g               = 9.81                          # gravitational acceleration [m/s²]
rho             = 1000.                         # density of water [kg/m³]

# define controller constants
target_level = 8.  # m
Kp = 0.1
Ti = 100.
deadband_range = 0.05 # m

# reservoir
initial_level               = target_level
initial_influx              = Q_nenn/2        # initial influx  of volume to the reservoir [m³/s]
initial_pressure_unit       = 'Pa'              # DO NOT CHANGE! for pressure conversion in print statements and plot labels 
conversion_pressure_unit    = 'bar'             # for pressure conversion in print statements and plot labels
area_base                   = 74.               # total base are of the cuboid reservoir [m²]   
area_outflux                = 1.                 # outflux area of the reservoir, given by pipeline area [m²]
critical_level_low          = 0.                # for yet-to-be-implemented warnings[m]
critical_level_high         = np.inf            # for yet-to-be-implemented warnings[m]

p0 = rho*g*initial_level-0.5*rho*(initial_influx/area_outflux)**2

# offset the pressure in front of the turbine to get realisitc fluxes
h_fict = 100
offset_pressure = rho*g*h_fict

t_max                       = 1e3 #s
nt                          = int(1e6)      # number of simulation steps of reservoir in between timesteps of pipeline              
dt                          = t_max/nt

t_vec = np.arange(0,nt+1,1)*dt



In [28]:
# create objects

V = Ausgleichsbecken_class(area_base,area_outflux,critical_level_low,critical_level_high,dt)
V.set_steady_state(initial_influx,initial_level,initial_pressure_unit,conversion_pressure_unit)

T1 = Francis_Turbine(Q_nenn,p_nenn)
T1.set_steady_state(initial_influx,p0+offset_pressure)
T1.set_closing_time(500)

Pegelregler = PI_controller_class(target_level,deadband_range,Kp,Ti,dt)

In [29]:
level_vec       = np.full(nt+1,V.level)
LA_ist_vec      = np.full(nt+1,T1.LA)
LA_soll_vec     = np.full(nt+1,T1.LA)
Q_vec           = np.full(nt+1,initial_influx)

Pegelregler.control_variable = T1.LA

In [30]:
# time loop

for i in range(nt+1):

    if np.mod(i,1e4) == 0:
        print(t_vec[i])

    if t_vec[i] == 0.4*np.max(t_vec):
        V.influx = 0

    p = rho*g*V.level-0.5*rho*(V.outflux_vel)**2

    LA_soll = Pegelregler.get_control_variable(V.level)
    T1.change_LA(LA_soll,dt)
    LA_soll_vec[i] = LA_soll
    LA_ist_vec[i]  = T1.LA
    Q_vec[i] = T1.get_Q(p+offset_pressure)

    V.pressure = p
    V.outflux_vel = 1/V.area_outflux*Q_vec[i]

    V.e_RK_4()                                                              
    V.level = V.update_level(V.timestep)                                    
    V.set_volume()                                                                                                                      
    level_vec[i] = V.level 
    
    

0.0
10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0
110.0
120.0
130.0
140.0
150.0
160.0
170.0
180.0
190.0
200.0
210.0
220.0
230.0
240.0
250.0
260.0
270.0
280.0
290.0
300.0
310.0
320.0
330.0
340.0
350.0
360.0
370.0
380.0
390.0
400.0
410.0
420.0
430.0
440.0
450.0
460.0
470.0
480.0
490.0
500.0
510.0
520.0
530.0
540.0
550.0
560.0
570.0
580.0
590.0
600.0
610.0
620.0
630.0
640.0
650.0
660.0
670.0
680.0
690.0
700.0
710.0
720.0
730.0
740.0
750.0
760.0
770.0
780.0
790.0
800.0
810.0
820.0
830.0
840.0
850.0
860.0
870.0
880.0
890.0
900.0
910.0
920.0
930.0
940.0
950.0
960.0
970.0
980.0
990.0
1000.0


In [31]:
%matplotlib qt5
# time loop

# create a figure and subplots to display the velocity and pressure distribution across the pipeline in each pipeline step
fig1,axs1 = plt.subplots(3,1)
axs1[0].set_title('Level')
axs1[0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs1[0].set_ylabel(r'$h$ [$\mathrm{m}$]')
axs1[0].plot(t_vec,level_vec)
axs1[0].set_ylim([0.85*initial_level,1.05*initial_level])
axs1[1].set_title('Flux')
axs1[1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs1[1].set_ylabel(r'$Q$ [$\mathrm{m} / \mathrm{s}^3$]')
axs1[1].plot(t_vec,Q_vec)
axs1[1].set_ylim([0,2*initial_influx])
axs1[2].set_title('LA')
axs1[2].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs1[2].set_ylabel(r'$LA$ [\%]')
axs1[2].plot(t_vec,LA_soll_vec)
axs1[2].plot(t_vec,LA_ist_vec)
axs1[2].set_ylim([0,1])
fig1.tight_layout()
fig1.show()
plt.pause(1)

In [32]:
fig2 = plt.figure()
plt.plot(t_vec,Pegelregler.error_history[1:])

In [33]:
print(level_vec[:])

[8.         8.         8.         ... 7.49916924 7.49916924 7.49916924]
